In [13]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [14]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prev_add = add!(model, AdditionNode())
    
    x_prior = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1   = add!(model, constvar(:add_1, 1.0))

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    
    add_x_and_noise = add!(model, AdditionNode())
    
    x = add!(model, simplerandomvar(:x))
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = add!(model, datavar(:y, Float64))
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(model, x_prev_add)
    activate!(model, add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getmarginal(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [15]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [19]:
@btime kalman($data)

  929.061 μs (14934 allocations: 648.66 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=2.314602396771967, σ=14.002800840280099)
 Normal{Float64}(μ=-0.0432330799734945, σ=9.950371902099892)
 Normal{Float64}(μ=-2.9318810219643043, σ=8.137884587711596)
 Normal{Float64}(μ=-3.1549322332810688, σ=7.053456158585984)
 Normal{Float64}(μ=-1.5655734926125944, σ=6.311944030978033)
 Normal{Float64}(μ=-1.1017514819323355, σ=5.763904177042351)
 Normal{Float64}(μ=-0.3097687207012802, σ=5.337605126836238)
 Normal{Float64}(μ=1.4790737893861803, σ=4.993761694389224)
 Normal{Float64}(μ=6.166962393857403, σ=4.708816093480111)
 Normal{Float64}(μ=8.36079961117114, σ=4.467670516087703)
 Normal{Float64}(μ=10.632594992771002, σ=4.2601432284230505)
 Normal{Float64}(μ=12.699187679724341, σ=4.079085082240021)
 Normal{Float64}(μ=13.869563800607509, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.9156598191099, σ=0.582706653680032)
 Normal{Float64}(μ=589.9399966942094, σ=0.5822126418123303)
 Normal{Float64}(μ=590.9556907199351, σ=0.58171988427037

In [18]:
@time kalman(data)

  0.001532 seconds (14.93 k allocations: 648.656 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=2.314602396771967, σ=14.002800840280099)
 Normal{Float64}(μ=-0.0432330799734945, σ=9.950371902099892)
 Normal{Float64}(μ=-2.9318810219643043, σ=8.137884587711596)
 Normal{Float64}(μ=-3.1549322332810688, σ=7.053456158585984)
 Normal{Float64}(μ=-1.5655734926125944, σ=6.311944030978033)
 Normal{Float64}(μ=-1.1017514819323355, σ=5.763904177042351)
 Normal{Float64}(μ=-0.3097687207012802, σ=5.337605126836238)
 Normal{Float64}(μ=1.4790737893861803, σ=4.993761694389224)
 Normal{Float64}(μ=6.166962393857403, σ=4.708816093480111)
 Normal{Float64}(μ=8.36079961117114, σ=4.467670516087703)
 Normal{Float64}(μ=10.632594992771002, σ=4.2601432284230505)
 Normal{Float64}(μ=12.699187679724341, σ=4.079085082240021)
 Normal{Float64}(μ=13.869563800607509, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.9156598191099, σ=0.582706653680032)
 Normal{Float64}(μ=589.9399966942094, σ=0.5822126418123303)
 Normal{Float64}(μ=590.9556907199351, σ=0.58171988427037

In [17]:
@time kalman(data)

  1.516477 seconds (3.63 M allocations: 188.712 MiB, 2.39% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=2.314602396771967, σ=14.002800840280099)
 Normal{Float64}(μ=-0.0432330799734945, σ=9.950371902099892)
 Normal{Float64}(μ=-2.9318810219643043, σ=8.137884587711596)
 Normal{Float64}(μ=-3.1549322332810688, σ=7.053456158585984)
 Normal{Float64}(μ=-1.5655734926125944, σ=6.311944030978033)
 Normal{Float64}(μ=-1.1017514819323355, σ=5.763904177042351)
 Normal{Float64}(μ=-0.3097687207012802, σ=5.337605126836238)
 Normal{Float64}(μ=1.4790737893861803, σ=4.993761694389224)
 Normal{Float64}(μ=6.166962393857403, σ=4.708816093480111)
 Normal{Float64}(μ=8.36079961117114, σ=4.467670516087703)
 Normal{Float64}(μ=10.632594992771002, σ=4.2601432284230505)
 Normal{Float64}(μ=12.699187679724341, σ=4.079085082240021)
 Normal{Float64}(μ=13.869563800607509, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.9156598191099, σ=0.582706653680032)
 Normal{Float64}(μ=589.9399966942094, σ=0.5822126418123303)
 Normal{Float64}(μ=590.9556907199351, σ=0.58171988427037